In [1]:
%cd drive/MyDrive/data/aT

/content/drive/MyDrive/data/aT


In [2]:
%ls

data/       scaled_data/   weights_1.pth  weights_3.pth  weights_5.pth
result.csv  weights_0.pth  weights_2.pth  weights_4.pth  weights_6.pth


# Import

In [3]:
import torch
from torch import nn, Tensor
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader

import os
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

np.random.seed(seed=2020320120)
torch.random.manual_seed(seed=2020320120)

# Dataset
데이터셋을 만들 때 고려해야 할 조건
* train, val, test를 어떻게 구분할 것인가? -> 먼저, split 인자를 통해 (train, val), test를 구분한다.
* 특히 train, val을 어떻게 구분할 것인가? -> phase 인자를 통해 train, val을 구분한다.
* 품목에 따라 데이터셋을 만들 것인가? -> use_all_items 인자와 item_idx 인자를 통해 모든 품목을 사용할 것인지, 하나의 품목만 사용할 것인지, 하나의 품목만 사용할 것이라면 사용할 품목의 인덱스는 무엇인지 설정한다.
* scaled_data를 사용한다.

In [4]:
def create_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

In [5]:
class NongDataset(Dataset):
    def __init__(self, split, use_all_items=True, item_idx=None):
        super().__init__()

        self.split = split # 'train' 또는 'test'; 'train'인 경우 self.phase 인자 필요
        self.use_all_items = use_all_items # 모든 품목 사용 여부
        self.item_idx = item_idx # 모든 품목을 사용하지 않을 경우, 사용할 품목 인덱스
        
        if self.split == 'train':
            self.init_train(use_all_items, item_idx)
        elif self.split == 'test':
            self.init_test(use_all_items, item_idx)
        else:
            raise ValueError()
    
    def init_train(self, use_all_items, item_idx):
        # 'train'인 경우, self.phase 인자 필요; 'train' or 'val'
        self.phase = 'train'

        # file name list; "train_0.csv"
        self.file_name_list = os.listdir('./scaled_data/train')

        # file path list; "./scaled_data/train/train_0.csv"
        if use_all_items: # 모든 품목을 사용하는 경우
            self.file_path_list = [f'./scaled_data/train/{file}' for file in self.file_name_list]
        else: # 한 가지 품목만 사용하는 경우
            for file_name in self.file_name_list:
                # 파일 품목 인덱스
                file_item_idx = int(file_name.split('_')[-1].split('.')[0])

                if file_item_idx == item_idx: # 사용할 품목 인덱스와 파일 품목 인덱스 동일
                    self.file_path_list = [f'./scaled_data/train/{file_name}']

        # 학습 데이터에서 사용하지 않을 열
        train_del_column_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ']

        # Dataset
        self.dataset = {
            'train': {
                'inputs': [],
                'labels': [],
            },
            'val': {
                'inputs': [],
                'labels': [],
            },
        }

        for file_path in self.file_path_list:
            # 판다스로 csv 파일 읽기
            data = pd.read_csv(file_path)

            # ' '을 np.nan으로 바꾸기
            for column in data.columns:
                data[column] = data[column].replace({' ': np.nan})
            
            # 학습 데이터에서 사용하지 않을 열 제거
            data.drop(train_del_column_list, axis=1, inplace=True)
            data.set_index('datadate', drop=True, inplace=True)

            # nan 값은 0으로 채우기
            data = data.fillna(0)

            # inputs와 labels 구분하기
            x, y = data[[i for i in data.columns if i != '해당일자_전체평균가격(원)']], data['해당일자_전체평균가격(원)']

            y = y[14:]

            # time window 만들기
            data_x = create_time_window(x, 13, 1) # 입력은 14일
            data_y = create_time_window(y, 27, 1) # 출력은 28일

            xdata = data_x[:len(data_y)]
            ydata = data_y

            x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

            self.dataset['train']['inputs'].append(x_train)
            self.dataset['train']['labels'].append(y_train)
            self.dataset['val']['inputs'].append(x_val)
            self.dataset['val']['labels'].append(y_val)
        
        self.dataset['train']['inputs'] = torch.tensor(np.concatenate(self.dataset['train']['inputs'], axis=0, dtype=np.float32))
        self.dataset['train']['labels'] = torch.tensor(np.concatenate(self.dataset['train']['labels'], axis=0, dtype=np.float32))
        self.dataset['val']['inputs'] = torch.tensor(np.concatenate(self.dataset['val']['inputs'], axis=0, dtype=np.float32))
        self.dataset['val']['labels'] = torch.tensor(np.concatenate(self.dataset['val']['labels'], axis=0, dtype=np.float32))

        print(self.dataset['train']['inputs'].shape)
        print(self.dataset['train']['labels'].shape)
        print(self.dataset['val']['inputs'].shape)
        print(self.dataset['val']['labels'].shape)

    def set_phase(self, phase):
        self.phase = phase

    def init_test(self, use_all_items, item_idx):
        # file path list; "./scaled_data/test/set_0/test_0.csv"
        self.file_path_list = []

        # for dir_name in os.listdir(f'./scaled_data/test'):
        #     for file_name in os.listdir(f'./scaled_data/test/{dir_name}'):
        #         file_path = f'./scaled_data/test/{dir_name}/{file_name}'

        #         if use_all_items: # 모든 품목을 사용하는 경우
        #             self.file_path_list.append(file_path)
        #         else: # 한 가지 품목만 사용하는 경우
        #             # 파일 품목 인덱스
        #             file_item_idx = int(file_name.split('_')[-1].split('.')[0])

        #             if file_item_idx == item_idx: # 인덱스 일치
        #                 self.file_path_list.append(file_path)

        for i in range(10):
            for j in range(37):
                file_path = f'./scaled_data/test/set_{i}/test_{j}.csv'

                if use_all_items:
                    self.file_path_list.append(file_path)
                else:
                    if j == item_idx:
                        self.file_path_list.append(file_path)

        # 테스트에서 사용하지 않을 열
        test_del_column_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)']

        check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월']

        zero_csv = [0 for i in range(14)]

        # Dataset
        self.dataset = []

        for idx, file_path in enumerate(self.file_path_list):
            # 판다스로 csv 파일 읽기
            data = pd.read_csv(file_path)

            # 
            if len(data) == 0:
                data['zero_non'] = zero_csv
                data = data.fillna(0)
                data.drop('zero_non', axis=1, inplace=True)
            
            file_number = file_path.split('/')[-1].split('_')[-1].split('.')[0]

            data.drop(test_del_column_list, axis=1, inplace=True)
            data.set_index('datadate', drop=True, inplace=True)
            
            add_col = [i for i in check_col if i not in data.columns]

            for a in add_col:
                data[a] = 0

            for a in data.columns:
                data[a] = data[a].replace({' ': np.nan})
            
            data = data.fillna(0)

            x_test = np.array(data, dtype=np.float32)

            x_test = x_test.reshape(1, x_test.shape[0], x_test.shape[1])

            if x_test.shape == (1, 14, 50):
                x_test = x_test[:, :, :49]

            self.dataset.append(x_test)

        self.dataset = torch.tensor(np.concatenate(self.dataset, axis=0, dtype=np.float32))

        print(self.dataset.shape)

    def __getitem__(self, idx):
        if self.split == 'train':
            return self.dataset[self.phase]['inputs'][idx], self.dataset[self.phase]['labels'][idx]
        elif self.split == 'test':
            return self.dataset[idx]
    
    def __len__(self):
        if isinstance(self.dataset, dict):
            return len(self.dataset[self.phase]['inputs'])
        else:
            return len(self.dataset)

# Network

In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [29]:
class Nongransformer(nn.Module):
    def __init__(self, d_model: int, nhead: int, nlayers: int, d_hid: int = 2048, dropout: float = 0.5):
        super().__init__()

        self.dim_mag = nn.Sequential(
            nn.Linear(49, d_model),
            nn.LeakyReLU(),
        )
        # self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.decoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(d_model * 14, d_model * 7),
            nn.LeakyReLU(),
            nn.Linear(d_model * 7, d_model * 7),
            nn.LeakyReLU(),
            nn.Linear(d_model * 7, 28),
        )

    def forward(self, src: Tensor) -> Tensor:
        src_mag = self.dim_mag(src)
        # src_pe = self.pos_encoder(src_mag)
        src_encode = self.transformer_encoder(src_mag)
        return self.decoder(src_encode)
        

# Train

In [26]:
train_dataset = NongDataset(split='train', use_all_items=True)
test_dataset = NongDataset(split='test', use_all_items=True)

torch.Size([41995, 14, 49])
torch.Size([41995, 28])
torch.Size([10508, 14, 49])
torch.Size([10508, 28])
torch.Size([370, 14, 49])


In [ ]:
train_dataset_0 = NongDataset(split='train', use_all_items=False, item_idx=0)

In [32]:
def train(epochs, train_dataset):
    # DataLoader
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=128,
        shuffle=True
    )

    for inputs, labels in train_dataloader:
        print(f'type(inputs): {type(inputs)}')
        print(f'type(labels): {type(labels)}')
        print(f'inputs.shape: {inputs.shape}')
        print(f'labels.shape: {labels.shape}')
        break

    # Net
    net = Nongransformer(
        d_model=512,
        nhead=8,
        nlayers=4,
    )

    for name, param in net.named_parameters():
        nn.init.normal_(param)
    
    # Criterion
    criterion = nn.L1Loss(reduction='mean')

    # Optimizer
    optimizer = optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.999))

    # Learning Rate Scheduler
    scheduler = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

    # Device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    net = net.to(device)

    best_loss = 987654321.0
    save_count = 0

    for epoch in range(epochs):
        print(f'{epoch} Epoch ' + '=' * 50)

        # Train
        train_dataloader.dataset.set_phase('train')
        net.train()

        train_loss = 0.0
        train_size = float(len(train_dataloader.dataset))
        print(f'train_size: {train_size}')

        for inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            n_samples = inputs.shape[0]

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = net(inputs)
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            scheduler.step()

            train_loss += loss.item() * n_samples
        
        print(f'Train Loss: {train_loss / train_size}')

        # Eval
        train_dataloader.dataset.set_phase('val')
        net.eval()

        eval_loss = 0.0
        eval_size = float(len(train_dataloader.dataset))
        print(f'eval_size: {eval_size}')

        for inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            n_samples = inputs.shape[0]

            with torch.set_grad_enabled(False):
                outputs = net(inputs)
                loss = criterion(outputs, labels)
            
            eval_loss += loss.item() * n_samples
        
        print(f'Eval Loss: {eval_loss / eval_size}')

        if eval_loss < best_loss:
            best_loss = eval_loss
            torch.save(net.state_dict(), f'./weights_0903_{save_count}.pth')
            save_count = save_count + 1
            print(f'Weight Save Count: {save_count}')

train(epochs=100, train_dataset=train_dataset)

type(inputs): <class 'torch.Tensor'>
type(labels): <class 'torch.Tensor'>
inputs.shape: torch.Size([128, 14, 49])
labels.shape: torch.Size([128, 28])
0 Epoch ==================================================
train_size: 41995.0


100%|██████████| 329/329 [00:23<00:00, 13.78it/s]


Train Loss: 57957.034410160435
eval_size: 10508.0


100%|██████████| 83/83 [00:01<00:00, 49.82it/s]


Eval Loss: 43719.255547856395
Weight Save Count: 1
1 Epoch ==================================================
train_size: 41995.0


...

Train Loss: 56125.31208439993
eval_size: 10508.0


100%|██████████| 83/83 [00:01<00:00, 50.52it/s]


Eval Loss: 43728.16370414208
8 Epoch ==================================================
train_size: 41995.0


 33%|███▎      | 107/329 [00:07<00:16, 13.66it/s]


KeyboardInterrupt: ignored

In [34]:
def test(test_dataset, weight_path):
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=37,
        shuffle=False,
    )

    net = Nongransformer(
        d_model=512,
        nhead=8,
        nlayers=4,
    )

    net.load_state_dict(torch.load(weight_path))

    for inputs in test_dataloader:
        print(type(inputs))
        print(inputs.shape)
        break

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    net = net.to(device)
    net.eval()

    result = []

    for inputs in test_dataloader:
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = net(inputs)
            outputs = outputs.T
            
            outputs_1 = outputs[:14, :]
            outputs_2 = outputs[14:, :].mean(dim=0).reshape(1, -1)
            final_outputs = torch.cat([outputs_1, outputs_2], dim=0)
        result.append(final_outputs)

    result = torch.cat(result, dim=0)
    print(f'result.shape: {result.shape}')

    result_np = result.cpu().detach().numpy()

    index = []
    for i in range(10):
        for _ in range(15):
            index.append(str(i))
    
    columns = []
    for i in range(37):
        columns.append(f'품목{i} 변동률')

    date = []
    for i in range(10):
        for j in range(14):
            date.append(f'd+{j+1}')
        date.append('d+22 ~ 28 평균')
    
    date_pd = pd.DataFrame(data=date, index=index, columns=['일자'])

    result_pd = pd.DataFrame(data=result_np, index=index, columns=columns)

    final_pd = pd.concat([date_pd, result_pd], axis=1)

    print(final_pd.head())

    final_pd.to_csv('./result_0903_0.csv', encoding='utf-8-sig')

# Train Execute
test(test_dataset, weight_path='./weights_0903_0.pth')

<class 'torch.Tensor'>
torch.Size([37, 14, 49])
result.shape: torch.Size([150, 37])
    일자       품목0 변동률       품목1 변동률       품목2 변동률       품목3 변동률       품목4 변동률  \
0  d+1  68551.109375  68551.210938  68551.148438  68550.890625  68551.375000   
0  d+2  80551.804688  80551.914062  80551.906250  80551.976562  80551.835938   
0  d+3   2291.870117   2291.799316   2291.870850   2292.195557   2291.647461   
0  d+4  23230.343750  23230.130859  23230.210938  23230.126953  23230.437500   
0  d+5   3696.071289   3696.183594   3696.156250   3696.082031   3696.394531   

        품목5 변동률       품목6 변동률       품목7 변동률       품목8 변동률  ...      품목27 변동률  \
0  68550.875000  68551.304688  68551.226562  68551.250000  ...  68551.343750   
0  80552.445312  80551.312500  80552.101562  80551.531250  ...  80551.890625   
0   2292.086426   2291.582031   2292.026367   2292.052979  ...   2291.705566   
0  23230.404297  23230.476562  23230.044922  23230.472656  ...  23230.330078   
0   3695.867188   3696.305664   369